##### Copyright 2020 The TensorFlow Authors.

In [25]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multi-task recommenders

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/examples/multitask"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/multitask.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/multitask.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/multitask.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In the [basic retrieval tutorial](basic_retrieval) we built a retrieval system using movie watches as positive interaction signals.

In many applications, however, there are multiple rich sources of feedback to draw upon. For example, an e-commerce site may record user visits to product pages (abundant, but relatively low signal), image clicks, adding to cart, and, finally, purchases. It may even record post-purchase signals such as reviews and returns.

Integrating all these different forms of feedback is critical to building systems that users love to use, and that do not optimize for any one metric at the expense of overall performance.

In addition, building a joint model for multiple tasks may produce better results than building a number of task-specific models. This is especially true where some data is abundant (for example, clicks), and some data is sparse (purchases, returns, manual reviews). In those scenarios, a joint model may be able to use representations learned from the abundant task to improve its predictions on the sparse task via a phenomenon known as [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). For example, [this paper](https://openreview.net/pdf?id=SJxPVcSonN) shows that a model predicting explicit user ratings from sparse user surveys can be substantially improved by adding an auxiliary task that uses abundant click log data.

In this tutorial, we are going to build a multi-objective recommender for Movielens, using both implicit (movie watches) and explicit signals (ratings).

## Imports


Let's first get our imports out of the way.


In [26]:
!pip install tensorflow==2.15.0
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

!pip show tensorflow-datasets
!pip show tensorflow-recommenders

ERROR: Operation cancelled by user
Name: tensorflow-datasets
Version: 4.9.8
Summary: tensorflow/datasets is a library of datasets ready to use with TensorFlow.
Home-page: https://github.com/tensorflow/datasets
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, array_record, dm-tree, etils, immutabledict, numpy, promise, protobuf, psutil, pyarrow, requests, simple_parsing, tensorflow-metadata, termcolor, toml, tqdm, wrapt
Required-by: 
Name: tensorflow-recommenders
Version: 0.7.3
Summary: Tensorflow Recommenders, a TensorFlow library for recommender systems.
Home-page: https://github.com/tensorflow/recommenders
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, tensorflow
Required-by: 


In [ ]:
!pip install numpy==1.25.2
!pip install --upgrade pandas

In [28]:
import os
import pprint
import tempfile
import pandas as pd

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## Preparing the dataset



In [29]:
pref = pd.read_csv('/content/unified_preferences.csv')

In [30]:
pref.dtypes

,0
user_id,int64
item_id,object
item_type,int64


In [31]:
pref.head()

,user_id,item_id,item_type
0,1,0140042393,1
1,1,0140121617,1
2,1,4oU7NbUt2VfN8GBT4hnsrX,3
3,1,3NLm801woJocONz1NmPJZR,3
4,1,5FVd6KXrgO9B3JPmC8OPst,3


In [32]:
pref_subset = pref[['item_id']].astype(str)
pref['user_id'] = pref['user_id'].astype(str)

In [33]:
pref_subset = pref_subset['item_id'].unique()

In [34]:
items_tf = tf.data.Dataset.from_tensor_slices({'item_id': pref_subset})
watched_tf = tf.data.Dataset.from_tensor_slices(dict(pref))

In [35]:
#ratings = tfds.load('movielens/100k-ratings', split="train")
#movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
watched = watched_tf.map(lambda x: {
    "item_id": x["item_id"],
    "user_id": x["user_id"],
    #"Book-Rating": x["Book-Rating"],
})

In [36]:
items = items_tf.map(lambda x:
    x["item_id"]
    #"Book-Title": x["Book-Title"]
)

And repeat our preparations for building vocabularies and splitting the data into a train and a test set:

In [37]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = watched.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(5_000)
test = shuffled.skip(5_000).take(2_000)

item_titles = items.batch(1_000)
user_ids = watched.batch(1_000).map(lambda x: x["user_id"])

unique_items = np.unique(np.concatenate(list(item_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## A multi-task model

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In this tutorial, we will define our models as before, but instead of having  a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

```python
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add 1 to account for the unknown token.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])
```

However, now we will have two tasks. The first is the rating task:

```python
tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)
```

Its goal is to predict the ratings as accurately as possible.

The second is the retrieval task:

```python
tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128)
    )
)
```

As before, this task's goal is to predict which movies the user will or will not watch.

### Putting it together

We put it all together in a model class.

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [38]:
@tf.keras.utils.register_keras_serializable()
class ItemModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float, unique_user_ids, unique_items) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32
    # Store values (optional, for rehydration or reference)
    self.unique_user_ids = unique_user_ids
    self.unique_items = unique_items


    # User and item models.
    self.item_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=list(self.unique_items), mask_token=None),
      tf.keras.layers.Embedding(len(self.unique_items) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=list(self.unique_user_ids), mask_token=None),
      tf.keras.layers.Embedding(len(self.unique_user_ids) + 1, embedding_dimension)
    ])

    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=items.batch(128).map(self.item_model)
        )
    )

    # The loss weights.
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the item features and pass them into the item model.
    item_embeddings = self.item_model(features["item_id"])

    return (
        user_embeddings,
        item_embeddings
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    #ratings = features.pop("Book-Rating")

    user_embeddings, item_embeddings = self(features)

    # We compute the loss for each task.

    retrieval_loss = self.retrieval_task(user_embeddings, item_embeddings)

    # And combine them using the loss weights.
    return (self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

The model does OK on predicting ratings (with an RMSE of around 1.11), but performs poorly at predicting which movies will be watched or not: its accuracy at 100 is almost 4 times worse than a model trained solely to predict watches.

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [39]:
@tf.keras.utils.register_keras_serializable()
class ItemModel(tfrs.models.Model):

    def __init__(self,
                 unique_user_ids: list,
                 unique_items: list,
                 items=None,
                 use_full_items=False,
                 rating_weight: float = 0.0,
                 retrieval_weight: float = 1.0) -> None:
        super().__init__()

        embedding_dimension = 32

        # Convert lists to strings for serialization purposes
        self.unique_user_ids = unique_user_ids
        self.unique_items = unique_items
        self.use_full_items = use_full_items
        self.retrieval_weight = retrieval_weight
        self.rating_weight = rating_weight

        # We'll handle items differently - don't store it as instance attribute
        # since it's not serializable

        # User and item models
        self.item_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_items, mask_token=None),
            tf.keras.layers.Embedding(len(unique_items) + 1, embedding_dimension)
        ])

        self.user_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])

        # Initialize retrieval_task conditionally
        if use_full_items and items is not None:
            self.retrieval_task = tfrs.tasks.Retrieval(
                metrics=tfrs.metrics.FactorizedTopK(
                    candidates=items.batch(128).map(self.item_model)
                )
            )
        else:
            # For inference only
            self.retrieval_task = tfrs.tasks.Retrieval()

    def get_config(self):
        # Return a serialized configuration of the model
        config = super().get_config()
        config.update({
            "unique_user_ids": self.unique_user_ids,
            "unique_items": self.unique_items,
            "use_full_items": self.use_full_items,
            "rating_weight": self.rating_weight,
            "retrieval_weight": self.retrieval_weight,
            # Don't include items as it's not serializable
        })
        return config

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        item_embeddings = self.item_model(features["item_id"])
        return user_embeddings, item_embeddings

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings, item_embeddings = self(features)
        retrieval_loss = self.retrieval_task(user_embeddings, item_embeddings)
        return self.retrieval_weight * retrieval_loss

In [40]:
#model = ItemModel(rating_weight=0.0, retrieval_weight=1.0)

# For training (full dataset)
training_model = ItemModel(
    unique_user_ids=unique_user_ids,
    unique_items=unique_items,
    items=items,  # full dataset
    use_full_items=True
)

training_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))




In [41]:
cached_train = train.shuffle(100_000).batch(8192)
cached_test = test.batch(4096)

In [42]:
training_model.fit(cached_train, epochs=3)
metrics = training_model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
#print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
1/1 [==============================] - 36s 36s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 2.0000e-04 - factorized_top_k/top_10_categorical_accuracy: 2.0000e-04 - factorized_top_k/top_50_categorical_accuracy: 6.0000e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0012 - loss: 42586.0430 - regularization_loss: 0.0000e+00 - total_loss: 42586.0430
Epoch 2/3
1/1 [==============================] - 35s 35s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0040 - factorized_top_k/top_50_categorical_accuracy: 0.0332 - factorized_top_k/top_100_categorical_accuracy: 0.0576 - loss: 42521.9297 - regularization_loss: 0.0000e+00 - total_loss: 42521.9297
Epoch 3/3
1/1 [==============================] - 15s 15s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categor

In [43]:
trained_user_embeddings, trained_item_embeddings = training_model({
      "user_id": np.array(["1"]),
      "item_id": np.array(["0140121617"])
  })
print(trained_item_embeddings)
print(trained_user_embeddings)

tf.Tensor(
[[-0.00239785 -0.02761246 -0.01660804 -0.02753646 -0.04912577  0.04428406
   0.014399    0.02750684  0.0329139  -0.02260855 -0.01016223  0.03615342
  -0.01892405  0.02523023  0.04826703  0.04650329  0.01225485  0.01716726
  -0.02881373  0.00595834  0.04112146 -0.03671833 -0.02983879  0.0080384
  -0.0480487  -0.01919887  0.03313004 -0.02151817  0.02951271 -0.02590203
   0.00819289  0.04969715]], shape=(1, 32), dtype=float32)
tf.Tensor(
[[-0.01306314 -0.06618251 -0.02249299 -0.08099803 -0.22704351 -0.26297805
  -0.28541386  0.30307496 -0.2662568  -0.16499323  0.2076087   0.20036869
  -0.22634329 -0.27316123  0.21303058 -0.13588628  0.30524945  0.11317477
  -0.0278797   0.29561886 -0.2670893   0.24566962  0.32810876 -0.29029286
   0.28002727  0.03262868 -0.00255473 -0.27647516  0.29343027  0.19817255
  -0.02580537 -0.11555012]], shape=(1, 32), dtype=float32)


In [44]:
# 1. Extract all the necessary components from the training model
user_weights = training_model.user_model.get_weights()
item_weights = training_model.item_model.get_weights()

# 2. Save vocabularies separately
import pickle
with open('user_vocab.pkl', 'wb') as f:
    pickle.dump(unique_user_ids, f)
with open('item_vocab.pkl', 'wb') as f:
    pickle.dump(unique_items, f)

# 3. Save weights separately
import numpy as np
np.save('user_weights.npy', user_weights)
np.save('item_weights.npy', item_weights)

print("Model components saved successfully!")


def load_inference_model():
    # Load vocabularies
    with open('user_vocab.pkl', 'rb') as f:
        user_vocab = pickle.load(f)
    with open('item_vocab.pkl', 'rb') as f:
        item_vocab = pickle.load(f)

    # Load weights
    user_weights = np.load('user_weights.npy', allow_pickle=True)
    item_weights = np.load('item_weights.npy', allow_pickle=True)

    # Create embedding lookup models
    embedding_dimension = 32

    user_model = tf.keras.Sequential([
        tf.keras.layers.StringLookup(vocabulary=user_vocab, mask_token=None),
        tf.keras.layers.Embedding(len(user_vocab) + 1, embedding_dimension)
    ])

    item_model = tf.keras.Sequential([
        tf.keras.layers.StringLookup(vocabulary=item_vocab, mask_token=None),
        tf.keras.layers.Embedding(len(item_vocab) + 1, embedding_dimension)
    ])

    # Build models
    user_model(np.array([user_vocab[0]]))
    item_model(np.array([item_vocab[0]]))

    # Set weights
    user_model.set_weights(user_weights)
    item_model.set_weights(item_weights)

    # Create a simple inference model function
    def get_embeddings(user_ids, item_ids):
        user_embs = user_model(user_ids)
        item_embs = item_model(item_ids)
        return user_embs, item_embs

    return get_embeddings, user_model, item_model

# Example usage
get_embeddings, user_model, item_model = load_inference_model()

# Test it
test_user_id = np.array(["1"])
test_item_id = np.array(["0140121617"])
user_emb, item_emb = get_embeddings(test_user_id, test_item_id)
print("User embedding:", user_emb)
print("Item embedding:", item_emb)

Model components saved successfully!
User embedding: tf.Tensor(
[[-0.01306314 -0.06618251 -0.02249299 -0.08099803 -0.22704351 -0.26297805
  -0.28541386  0.30307496 -0.2662568  -0.16499323  0.2076087   0.20036869
  -0.22634329 -0.27316123  0.21303058 -0.13588628  0.30524945  0.11317477
  -0.0278797   0.29561886 -0.2670893   0.24566962  0.32810876 -0.29029286
   0.28002727  0.03262868 -0.00255473 -0.27647516  0.29343027  0.19817255
  -0.02580537 -0.11555012]], shape=(1, 32), dtype=float32)
Item embedding: tf.Tensor(
[[-0.00239785 -0.02761246 -0.01660804 -0.02753646 -0.04912577  0.04428406
   0.014399    0.02750684  0.0329139  -0.02260855 -0.01016223  0.03615342
  -0.01892405  0.02523023  0.04826703  0.04650329  0.01225485  0.01716726
  -0.02881373  0.00595834  0.04112146 -0.03671833 -0.02983879  0.0080384
  -0.0480487  -0.01919887  0.03313004 -0.02151817  0.02951271 -0.02590203
   0.00819289  0.04969715]], shape=(1, 32), dtype=float32)


In [45]:
# Export item embeddings with their corresponding item-id
item_embeddings = item_model.get_weights()[0]
# Export user embeddings with their corresponding user-id
user_embeddings = user_model.get_weights()[0]


# export
import numpy as np
np.save('item_embeddings.npy', item_embeddings)
np.save('user_embeddings.npy', user_embeddings)

In [50]:
# prompt: Export the user embedding along with the user id

# Export user embeddings with their corresponding user-id
user_embeddings = user_model.get_weights()[0]
item_embeddings = item_model.get_weights()[0]

# Assuming you have a mapping from index to user ID (from your StringLookup layer)
# Replace this with your actual mapping if it's different
user_id_mapping = {i: user_id for i, user_id in enumerate(unique_user_ids)}
item_id_mapping = {i: item_id for i, item_id in enumerate(unique_items)}

# Create a DataFrame to store the user embeddings and their IDs

# Build DataFrame
user_df = pd.DataFrame(user_embeddings)
user_df['user_id'] = user_df.index.map(user_id_mapping)

item_df = pd.DataFrame(item_embeddings)
item_df['item_id'] = item_df.index.map(item_id_mapping)
# Save to CSV
user_df.to_csv('user_embeddings.csv', index=False)
item_df.to_csv('item_embeddings.csv', index=False)


### Joint model

Let's now train a model that assigns positive weights to both tasks.

The result is a model that performs roughly as well on both tasks as each specialized model.

### Making prediction

We can use the trained multitask model to get trained user and movie embeddings, as well as the predicted rating:

While the results here do not show a clear accuracy benefit from a joint model in this case, multi-task learning is in general an extremely useful tool. We can expect better results when we can transfer knowledge from a data-abundant task (such as clicks) to a closely related data-sparse task (such as purchases).

In [ ]:
# 📌 Import required libraries
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# 1. Prepare user IDs and track IDs
# Drop duplicate users to ensure one track per user
user_rate_map = ratings_df.drop_duplicates(subset=["User-ID"])
all_user_ids = user_rate_map["User-ID"].values
all_book_ids = user_rate_map["ISBN"].values

# 2. Pass user IDs and track IDs into the model to obtain user embeddings
track_embeddings, user_embeddings, rating_predictions = model({
    "User-ID": all_user_ids,
    "ISBN": all_book_ids
})

# 3. Convert user embeddings to a NumPy array
user_embeddings_np = np.array(user_embeddings)

# 4. Compute cosine similarity between all user embeddings
similarity_matrix = cosine_similarity(user_embeddings_np)

# 5. Create a DataFrame to map user IDs to each other based on similarity
similarity_df = pd.DataFrame(similarity_matrix, index=all_user_ids, columns=all_user_ids)

# 6. Find the top 5 most similar users to a specific user
target_user_id = "276726"

# Sort the similarities in descending order (highest similarity first)
similar_users = similarity_df.loc[target_user_id].sort_values(ascending=False)

# Exclude the user themselves (the first entry) and select the top 5
top5_similar_users = similar_users[1:6]

# 7. Print the result
print(f"\n Top 5 users most similar to {target_user_id}:")
print(top5_similar_users)